In [6]:
import os
import warnings
warnings.filterwarnings("ignore")

# Only install faiss-cpu (skip faiss-gpu)
!pip install langchain pypdf chromadb sentence-transformers langchain_community rank_bm25 faiss-cpu
!pip install -U langchain-huggingface
!pip install pymupdf

# Now import
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from rank_bm25 import BM25Okapi
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import textwrap
from IPython.display import Markdown, display

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
import textwrap
from sklearn.metrics.pairwise import cosine_similarity


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 57.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2025-04-20 11:24:12.353826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745148252.574168      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745148252.636768      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ImportError: cannot import name 'set_config_context' from 'langchain_core.runnables.config' (/usr/local/lib/python3.11/dist-packages/langchain_core/runnables/config.py)

In [ ]:
!pip install torch
import torch  # Import the torch module

Uncomment the cells below if you're not using Groq API

In [ ]:
# model_name = "meta-llama/Llama-3.1-8B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# print(f"Model dtype: {model.dtype}")
# total_params = sum(p.numel() for p in model.parameters())
# print(f"Total Parameters: {total_params / 1e6} million")

# generator = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     return_full_text=False,
#     max_new_tokens=1000,
#     do_sample=True,
#     temperature=0.8,
#     top_p=0.9
# )


In [2]:
import os
from langchain.schema import Document

markdown_folder = '/kaggle/input/markdowns'  # Path to the folder with markdown files
all_docs = []

for md_file in os.listdir(markdown_folder):
    if md_file.endswith('.md'):
        md_path = os.path.join(markdown_folder, md_file)

        with open(md_path, 'r', encoding='utf-8') as file:
            content = file.read()

        metadata = {
            "source": md_file
        }

        all_docs.append(Document(page_content=content, metadata=metadata))

print(f"Total markdown files loaded: {len(all_docs)}")


Total markdown files loaded: 61


Uncomment the cell below if you're loading PDFs and parsing them on the go. (Not recommended)

In [7]:
# import os
# import fitz  # PyMuPDF
# from langchain.schema import Document

# pdf_folder = '/kaggle/input/women-laws-pak'  # Path to the folder
# all_pages = []

# for pdf_file in os.listdir(pdf_folder):
#     if pdf_file.endswith('.pdf'):
#         pdf_path = os.path.join(pdf_folder, pdf_file)

#         doc = fitz.open(pdf_path)
#         for page in doc:
#             text = page.get_text("text")
#             metadata = {
#                 "source": pdf_file,
#                 "page": page.number,
#                 "total_pages": len(doc)
#             }
#             all_pages.append(Document(page_content=text, metadata=metadata))

#         doc.close()

# print(f"Total pages loaded: {len(all_pages)}")


Total pages loaded: 714


In [ ]:
from langchain.text_splitter import TokenTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [ ]:

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1024,
#     chunk_overlap=64, 
#     separators=["\n\n", "\n", " ", ""]
# )


# text_splitter = SentenceTransformersTokenTextSplitter(
#     chunk_size=1024,
#     chunk_overlap=64,
#     model_name="sentence-transformers/all-MiniLM-L6-v2"
# )

# chunks = text_splitter.split_documents(all_pages)

# texts = [chunk.page_content for chunk in chunks]
# metadata = [chunk.metadata for chunk in chunks]

In [ ]:
# # from langchain.text_splitter import CharacterTextSplitter

# # text_splitter = CharacterTextSplitter(
# #     separator="\n",  # or just " " if needed
# #     chunk_size=256,
# #     chunk_overlap=30
# # )
# # chunks = text_splitter.split_documents(all_pages)

# import nltk

# # Now force clean download
# nltk.download('punkt_tab')

# from langchain.text_splitter import NLTKTextSplitter

# # Create the NLTK-based splitter
# text_splitter = NLTKTextSplitter(
#     chunk_size=1345,
#     chunk_overlap=42
# )

# # Split your documents (all_pages must be a list of Document objects)
# chunks = text_splitter.split_documents(all_pages)

# from langchain.text_splitter import TokenTextSplitter

# # Create the TokenTextSplitter
# text_splitter = TokenTextSplitter(
#     chunk_size=2048,       # Number of tokens per chunk
#     chunk_overlap=200       # Overlap in tokens between chunks
# )

# # Split your documents (all_pages must be a list of Document objects)
# chunks = text_splitter.split_documents(all_pages)


In [ ]:
# from langchain.embeddings import HuggingFaceEmbeddings

# embedding_model = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
#     encode_kwargs={"normalize_embeddings": True}
# )


# vectordb = FAISS.from_texts(texts, embedding_model, metadatas=metadata)
# vectordb.save_local("women_laws_faiss_index")

# num_embeddings = len(vectordb.index.reconstruct_n(0, vectordb.index.ntotal))
# print(f"Number of embeddings in vector store: {num_embeddings}")
# print(f"Number of text chunks processed: {len(texts)}")

# assert num_embeddings == len(texts), "Mismatch between embeddings and text chunks"

# tokenized_texts = [text.split() for text in texts]
# bm25 = BM25Okapi(tokenized_texts)


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Function for chunking and splitting documents
def chunk_and_split(all_docs, chunk_size, overlap=64):
    global embedding_model, vectordb, texts, metadata, bm25
    text_splitter = SentenceTransformersTokenTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    
    chunks = text_splitter.split_documents(all_docs)
    
    texts = [chunk.page_content for chunk in chunks]
    metadata = [chunk.metadata for chunk in chunks]
    
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
        encode_kwargs={"normalize_embeddings": True}
    )
    
    vectordb = FAISS.from_texts(texts, embedding_model, metadatas=metadata)
    vectordb.save_local(f"women_laws_faiss_index_{chunk_size}")
    
    num_embeddings = len(vectordb.index.reconstruct_n(0, vectordb.index.ntotal))
    print(f"Number of embeddings in vector store for chunk size {chunk_size}: {num_embeddings}")
    print(f"Number of text chunks processed for chunk size {chunk_size}: {len(texts)}")
    
    assert num_embeddings == len(texts), f"Mismatch between embeddings and text chunks for chunk size {chunk_size}"
    
    tokenized_texts = [text.split() for text in texts]
    bm25 = BM25Okapi(tokenized_texts)
    

In [ ]:
def linear_score_fusion(results_bm25, results_embedding, alpha=0.5):
    scores = {}
    for doc, score in results_bm25:
        doc_id = doc.page_content
        scores[doc_id] = scores.get(doc_id, 0) + alpha * score

    for doc, score in results_embedding:
        doc_id = doc.page_content
        scores[doc_id] = scores.get(doc_id, 0) + (1 - alpha) * score

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

def reciprocal_rank_fusion(results_bm25, results_embedding):
    scores = {}

    for rank, (doc, score) in enumerate(results_bm25):
        doc_id = doc.page_content
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank + 1)

    for rank, (doc, score) in enumerate(results_embedding):
        doc_id = doc.page_content
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (rank + 1)

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

def mmr_rerank(query_embedding, doc_embeddings, docs, lambda_param=0.5, k=5):
    selected = []
    selected_indices = []
    doc_indices = list(range(len(docs)))
    doc_embeddings = np.array(doc_embeddings)
    query_embedding = np.array(query_embedding).reshape(1, -1)

    similarity_to_query = cosine_similarity(doc_embeddings, query_embedding).flatten()
    similarity_between_docs = cosine_similarity(doc_embeddings)

    for _ in range(k):
        mmr_scores = []
        for idx in doc_indices:
            if idx in selected_indices:
                continue
            if not selected:
                diversity_penalty = 0
            else:
                diversity_penalty = max(similarity_between_docs[idx][j] for j in selected_indices)
            mmr_score = lambda_param * similarity_to_query[idx] - (1 - lambda_param) * diversity_penalty
            mmr_scores.append((idx, mmr_score))

        selected_idx = max(mmr_scores, key=lambda x: x[1])[0]
        selected.append(docs[selected_idx])
        selected_indices.append(selected_idx)

    return selected

def retrieve(query, k, summarize=True, lambda_param=0.5):
    # Dense retrieval: Get top-k similar docs + their embeddings
    query_embedding = embedding_model.embed_query(query)
    results_embedding = vectordb.similarity_search_with_score(query, k=24)  # larger k for reranking

    dense_docs = [doc for doc, _ in results_embedding]
    dense_embeddings = embedding_model.embed_documents([doc.page_content for doc in dense_docs])

    # MMR Reordering
    mmr_selected_docs = mmr_rerank(query_embedding, dense_embeddings, dense_docs, lambda_param=lambda_param, k=k)
    return mmr_selected_docs

# The retrieve function below is used for hybrid approach

# def retrieve(query, k=5):
#     # Dense retrieval
#     query_embedding = embedding_model.embed_query(query)
#     results_embedding = vectordb.similarity_search_with_score_by_vector(query_embedding, k=k)

#     # Sparse retrieval (BM25)
#     results_bm25 = [(idx, bm25.get_scores(query.split())[idx]) for idx in range(len(texts))]
#     results_bm25 = sorted(results_bm25, key=lambda x: x[1], reverse=True)[:k]
#     results_bm25_docs = [(Document(page_content=texts[idx], metadata=metadata[idx]), score)
#                          for idx, score in results_bm25]

#     # Reciprocal Rank Fusion (no re-trimming inside)
#     fused_results = reciprocal_rank_fusion(results_bm25_docs, results_embedding)

#     # Lookup dictionary to map back to full Document objects
#     doc_lookup = {doc.page_content: doc for doc, _ in results_bm25_docs + results_embedding}

#     # Return top-k fused documents
#     return [doc_lookup[doc_id] for doc_id, _ in fused_results if doc_id in doc_lookup][:k]


In [ ]:
eval_data = []
# List of relevant answers in the same order as the original questions
relevant_answers = [

    """Under the Zainab Alert, Response and Recovery Act, 2020, “abduction” includes:
    - Inducing a child under the age of 18 by deceit, force, or threat to move from one place to another, or to do any act.
    - Taking or detaining a child unlawfully from the lawful custody of a parent or legal guardian.
    - It includes the act of kidnapping as defined under the Pakistan Penal Code, 1860.""",

    """The Bonded Labour System (Abolition) Act of 1992 abolished all forms of bonded labour in Pakistan. Specifically, Section 6 of the Act states that any existing bonded debts at the time of the Act's commencement are extinguished. This means that bonded labourers are no longer obligated to repay such debts, and any agreements or customs enforcing bonded labour are rendered null and void.""",

    """Under Section 15 of the Bonded Labour System (Abolition) Act, Vigilance Committees are established to ensure the implementation of the Act. These committees are responsible for:
    - Monitoring the presence of bonded labour in their respective areas.
    - Rehabilitating freed bonded labourers.
    - Ensuring that the provisions of the Act are enforced effectively.

    They serve as a bridge between the government and the community to eradicate bonded labour practices.""",

    """In Pakistan, women's inheritance rights are protected under Islamic law, which mandates that female heirs receive their due shares of inheritance. The Supreme Court of Pakistan has emphasized that any attempt to deprive women of their inheritance rights is a violation of Islamic principles and the law. The Court has ruled that properties of deceased individuals vest immediately in their legal heirs upon death, and any denial of this right, especially to female heirs, is unacceptable.""",

    """Pakistan has enacted laws to address domestic violence, with specific legislation varying by province. For instance, the Punjab Protection of Women Against Violence Act 2016 provides mechanisms for protection and redress for victims of domestic abuse. Penalties for perpetrators can include imprisonment, fines, and restraining orders. However, enforcement remains a challenge, and advocacy continues for more robust implementation of these laws to protect women effectively.""",

    """Child marriage laws in Pakistan vary by region:
    - Sindh: The Sindh Child Marriage Restraint Act 2013 sets the minimum marriage age at 18 for both males and females. Violations can result in imprisonment and fines.
    - Punjab: The Punjab Marriage Restraint Act 2015 maintains the minimum marriage age at 16 for females and 18 for males. Efforts are ongoing to raise the minimum age for females to 18.
    - Khyber Pakhtunkhwa and Balochistan: These provinces still follow the Child Marriage Restraint Act of 1929, which sets the minimum marriage age at 16 for females.

    Despite these laws, child marriages persist, particularly in rural areas, due to cultural practices and lack of enforcement.""",


    """The medical examination must be conducted by a registered medical practitioner—preferably a female if the victim is female—immediately after the offence.
    Consent of the victim or their guardian must be obtained. The report should include the victim’s details, injuries, mental condition, materials collected for DNA, and exact timing of the examination.
    DNA samples must be collected with consent and sent promptly to a forensic lab while ensuring confidentiality."""



]

for i in range(len(eval_data)):
    eval_data[i]["ground_truth"] = relevant_answers[i]


In [ ]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def rerank_documents(question, docs, top_k=5):
    pairs = [[question, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    scored_docs = list(zip(docs, scores))
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in scored_docs[:top_k]]


In [ ]:
import time
import requests
from IPython.display import Markdown
import re
import torch


# Constants
GROQ_API_KEY = "you_api_key"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

total_retrieval_time = 0
total_generation_time = 0
eval_data = []

def call_groq(prompt, modelName):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": modelName,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.6
    }

    while True:
        response = requests.post(GROQ_URL, headers=headers, json=payload)

        if response.status_code == 429:
            retry_after = response.headers.get("Retry-After")
            wait = int(retry_after) if retry_after else 60
            print(f"🚫 Rate limit hit. Retrying in {wait} seconds...")
            time.sleep(wait + 1)
            continue

        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

def ask_question(question, k, generation_model, judge_model, chunk_size, chunk_overlap):
    global total_retrieval_time, total_generation_time

    start_retrieval = time.time()
    docs = retrieve(question, k=k)
    retrieval_duration = time.time() - start_retrieval
    docs = rerank_documents(question, docs, top_k=k)
    total_retrieval_time += retrieval_duration

    combined_context = ""
    for i, doc in enumerate(docs):
        page = doc.metadata.get("page", "?")
        combined_context += f"\nDocument {i+1} (Page {page}):\n{doc.page_content}\n"

    print("\n🔍 Top Retrieved Documents:\n")
    for i, doc in enumerate(docs):
        page = doc.metadata.get("page", "?")
        print(f"Document {i+1} (Page {page}): {doc.page_content[:500]}...\n{'-'*50}")

    start_generation = time.time()
    prompt = f"""Answer the following question based on the legal and social context provided below, which may include topics such as women's rights, juvenile justice, child protection, bonded labor, and labor laws in Pakistan.

{combined_context}

Question: {question}

Instructions:
    - Ensure your answer is grounded in the context
    - Use clear, factual language
    - Format using bullet points or headings if needed"""
    response = call_groq(prompt, generation_model)
    generation_duration = time.time() - start_generation
    total_generation_time += generation_duration

    # Clear GPU memory if needed
    torch.cuda.empty_cache()

    print("\n==============================")
    print("🧠 Question:")
    print(question)
    print("\n💬 Generated Answer:")
    print(response)
    print("==============================")
    print(f"\n⏱ Retrieval Time: {retrieval_duration:.2f}s | Generation Time: {generation_duration:.2f}s")

    # LLM Judge Evaluation
    judge_prompt = f"""
You are a strict evaluator. Given the context and the generated answer, assess the following:
1. *Faithfulness*: Is the answer factually accurate based on the context provided?
2. *Relevance*: Does the answer directly address the question using only the context?

Respond with:
- Faithfulness Score (0-5):
- Relevance Score (0-5):
- Justification:

### Context:
{combined_context}

### Question:
{question}

### Answer:
{response}
    """
    judgment = call_groq(judge_prompt, judge_model)
    print("\n📊 LLM Evaluation:\n", judgment)

    faithfulness_score = None
    relevance_score = None
    
    faithfulness_match = re.search(
        r"Faithfulness(?: Score)?[^0-9]*([\d.]+)\s*/?\s*5", judgment, re.IGNORECASE
    )
    relevance_match = re.search(
        r"Relevance(?: Score)?[^0-9]*([\d.]+)\s*/?\s*5", judgment, re.IGNORECASE
    )
    
    if faithfulness_match:
        faithfulness_score = float(faithfulness_match.group(1))  # Allow decimal scores
    if relevance_match:
        relevance_score = float(relevance_match.group(1))  # Allow decimal scores
    
    if faithfulness_score is None or relevance_score is None:
        print("⚠️ Could not parse evaluation scores!")


    eval_data.append({
        "chunk_size": chunk_size,
        "chunk_overlap": chunk_overlap,
        "question": question,
        "answer": response,
        "contexts": [doc.page_content for doc in docs],
        "retrieval_time": retrieval_duration,
        "generation_time": generation_duration,
        "total_time": retrieval_duration + generation_duration,
        "k": k,
        "generation_model": generation_model,
        "judge_model": judge_model,
        "llm_judgment": judgment,
        "faithfulness_score": faithfulness_score,
        "relevance_score": relevance_score
    })


The cell below was used when using Hugging Face models instead of Groq Inference API

In [ ]:
# import time
# import torch

# total_retrieval_time = 0
# total_generation_time = 0
# eval_data = []

# # Limit the maximum length of the combined context
# max_context_length = 2000  # Define a maximum context length to reduce GPU load

# def ask_question(question):
#     global total_retrieval_time, total_generation_time

#     # Record the start time for retrieval
#     start_retrieval = time.time()
#     docs = retrieve(question)
#     end_retrieval = time.time()
#     retrieval_duration = end_retrieval - start_retrieval
#     total_retrieval_time += retrieval_duration

#     # Combine all documents into one context, with truncation
#     combined_context = ""
#     for i, doc in enumerate(docs[:3]):
#         page_num = doc.metadata.get('page', '?')
#         content = doc.page_content[:500]  # Truncate each document to 500 characters
#         combined_context += f"\nDocument {i+1} (Page {page_num}):\n{content}\n"
#         if len(combined_context) > max_context_length:
#             break  # Stop appending if we've reached the maximum context length

#     print("\n🔍 Top Retrieved Documents:\n")
#     for i, doc in enumerate(docs[:3]):  # Limit to 3 documents for display
#         page_num = doc.metadata.get('page', '?')
#         print(f"Document {i+1} (Page {page_num}): {doc.page_content[:500]}...\n{'-'*50}")

#     # Generate answer
#     start_generation = time.time()
#     prompt = f"""Answer the following question based on the legal and social context provided below, which may include topics such as women's rights, juvenile justice, child protection, bonded labor, and labor laws in Pakistan.

#     {combined_context}

#     Question: {question}

#     Instructions:
#     - Ensure your answer is grounded in the context
#     - Use clear, factual language
#     - Format using bullet points or headings if needed"""
#     response = generator(
#         [{"role": "user", "content": prompt}],
#         max_new_tokens=250  # Adjust this to desired word/token limit
#     )[0]["generated_text"]

#     end_generation = time.time()
#     generation_duration = end_generation - start_generation
#     total_generation_time += generation_duration

#     # Clear GPU memory after generation
#     torch.cuda.empty_cache()

#     # Display the result
#     display(Markdown(f"**Question:** {question}"))
#     display(Markdown(f"**Answer:** {response}"))

#     print(f"\n⏱️ Retrieval Time: {retrieval_duration:.2f}s | Generation Time: {generation_duration:.2f}s")

#     # LLM Judge Evaluation
#     judge_prompt = f"""
# You are a strict evaluator. Given the context and the generated answer, assess the following:
# 1. **Faithfulness**: Is the answer factually accurate based on the context provided?
# 2. **Relevance**: Does the answer directly address the question using only the context?

# Respond with:
# - Faithfulness Score (0-5):
# - Relevance Score (0-5):
# - Justification:

# ### Context:
# {combined_context}

# ### Question:
# {question}

# ### Answer:
# {response}
# """

#     with torch.inference_mode():
#       judgment = generator([{"role": "user", "content": judge_prompt}])[0]["generated_text"]

#     print("\n📊 LLM Evaluation:\n", judgment)

#     # Save evaluation data
#     eval_data.append({
#         "question": question,
#         "answer": response,
#         "relevant_answer": combined_context,
#         "contexts": [doc.page_content for doc in docs],
#         "llm_judgment": judgment
#     })




In [ ]:
questions = [
    "What does abduction mean under the law of Pakistan?",
    "How does the Bonded Labour System Act protect laborers from debt bondage?",
    "What role do Vigilance Committees play according to the Act?",
    "What initiatives are there to protect women's entitlement to inheritance?",
    "What legal measures and punishments exist in Pakistan to address domestic violence against women?",
    "What are the current legal provisions and age restrictions regarding child marriage in Pakistan?",
    "What are the requirements for medical examination and DNA testing of a rape or sexual abuse victim?",
]

# Define your models
generation_model = "meta-llama/llama-4-maverick-17b-128e-instruct"
judge_model = "deepseek-r1-distill-llama-70b"

# Parameter sets
chunk_sizes = [256, 512, 1024, 2048]
chunk_overlaps = [32, 64, 128, 128]
k_values = [4, 6, 8]

for chunk_size, chunk_overlap in zip(chunk_sizes, chunk_overlaps):
    print(f"\n🚀 Processing with chunk size: {chunk_size}, overlap: {chunk_overlap}")
    
    # Chunk and split the documents
    chunk_and_split(all_docs, chunk_size, chunk_overlap)

    for k in k_values:
        print(f"\n🔍 Evaluating with top-{k} retrieved documents...")

        for q in questions:
            ask_question(
                question=q,
                k=k,
                generation_model=generation_model,
                judge_model=judge_model,
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap
            )
            display(Markdown(f"**k = {k} | chunk size = {chunk_size}, overlap = {chunk_overlap}**"))
            display(Markdown("---"))



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Make sure eval_data exists and is not empty
if "eval_data" in globals() and eval_data:
    df = pd.DataFrame(eval_data)

    # Convert columns to numeric if needed
    df["faithfulness_score"] = pd.to_numeric(df["faithfulness_score"], errors="coerce")
    df["relevance_score"] = pd.to_numeric(df["relevance_score"], errors="coerce")

    # Group by chunk_size, overlap, and k
    grouped = df.groupby(["chunk_size", "chunk_overlap", "k"])[
        ["retrieval_time", "generation_time", "faithfulness_score", "relevance_score"]
    ].mean().reset_index()

    print("\n📊 Summary grouped by chunk size, overlap, and k:")
    print(grouped)

    # Create output directory
    os.makedirs("rag_eval_charts", exist_ok=True)

    # ------------------- Lineplot: Faithfulness & Relevance vs Chunk Size -------------------
    plt.figure(figsize=(14, 6))

    # Faithfulness
    plt.subplot(1, 2, 1)
    sns.lineplot(data=grouped, x="chunk_size", y="faithfulness_score", hue="k", marker="o")
    plt.title("Faithfulness vs Chunk Size")
    plt.xlabel("Chunk Size")
    plt.ylabel("Faithfulness Score")
    plt.grid(True)

    # Relevance
    plt.subplot(1, 2, 2)
    sns.lineplot(data=grouped, x="chunk_size", y="relevance_score", hue="k", marker="o")
    plt.title("Relevance vs Chunk Size")
    plt.xlabel("Chunk Size")
    plt.ylabel("Relevance Score")
    plt.grid(True)

    plt.tight_layout()
    plt.savefig("rag_eval_charts/lineplot_faithfulness_relevance_vs_chunk_size.png")
    plt.show()

    # ------------------- Heatmaps: Chunk Size × k -------------------
    for metric in ["faithfulness_score", "relevance_score"]:
        pivot = grouped.pivot_table(index="chunk_size", columns="k", values=metric)
        plt.figure(figsize=(8, 6))
        sns.heatmap(pivot, annot=True, fmt=".2f", cmap="YlGnBu")
        title = f"{metric.replace('_', ' ').title()} Heatmap (Chunk Size × k)"
        plt.title(title)
        plt.ylabel("Chunk Size")
        plt.xlabel("k")

        filename = f"rag_eval_charts/heatmap_{metric}.png"
        plt.savefig(filename)
        plt.show()

else:
    print("⚠️ `eval_data` is empty or not defined.")


In [ ]:
# def print_total_times(num_queries):
#     # Display total times after all questions
#     print(f"\n🔢 Total Retrieval Time: {total_retrieval_time:.2f} seconds")
#     print(f"🔢 Total Generation Time: {total_generation_time:.2f} seconds")
#     print(f"🔢 Total Time (Retrieval + Generation): {total_retrieval_time + total_generation_time:.2f} seconds")

#     # Display average times per query
#     if num_queries > 0:
#         avg_retrieval_time = total_retrieval_time / num_queries
#         avg_generation_time = total_generation_time / num_queries
#         avg_total_time = (total_retrieval_time + total_generation_time) / num_queries

#         print(f"\n📊 Average Retrieval Time: {avg_retrieval_time:.2f} seconds/query")
#         print(f"📊 Average Generation Time: {avg_generation_time:.2f} seconds/query")
#         print(f"📊 Average Total Time: {avg_total_time:.2f} seconds/query")
#     else:
#         print("\n⚠️ No queries processed.")

# print_total_times(len(eval_data))


In [ ]:
# import re

# faithfulness_scores = []
# relevance_scores = []

# for i, entry in enumerate(eval_data):
#     judgment = entry.get("llm_judgment", "")

#     # Flexible regex to extract scores
#     faithfulness_match = re.search(r"Faithfulness.*?(\d+)", judgment, re.IGNORECASE)
#     relevance_match = re.search(r"Relevance.*?(\d+)", judgment, re.IGNORECASE)

#     if faithfulness_match and relevance_match:
#         faithfulness_scores.append(int(faithfulness_match.group(1)))
#         relevance_scores.append(int(relevance_match.group(1)))
#     else:
#         print(f"⚠️ Judgment {i+1} did not contain extractable scores:\n{judgment}\n{'-'*60}")

# # Print average if we have any valid scores
# if faithfulness_scores and relevance_scores:
#     avg_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)
#     avg_relevance = sum(relevance_scores) / len(relevance_scores)

#     print("\n📈 **Combined Evaluation Summary**")
#     print(f"- Average Faithfulness Score: {avg_faithfulness:.2f}/5")
#     print(f"- Average Relevance Score: {avg_relevance:.2f}/5")
# else:
#     print("❌ No valid scores found. Check the format of LLM judgments.")
